In [ ]:
# import geopandas as gpd
import pandas as pd
# from shapely.geometry import Point
import json

# Load the state polygons (GeoJSON with Kogi, Benue, Taraba)
# states_gdf = gpd.read_file("nigeria-states.geojson")

# Load your GeoJSON
with open("nigeria_lga.geojson") as f:
    states_gdf = json.load(f)

# Ensure CRS is WGS84 (latitude/longitude)
# states_gdf = states_gdf.to_crs("EPSG:4326")

# Load XYZ data
xyz_df = pd.read_csv("nga_pd_2020_1km_UNadj_ASCII_XYZ.csv")
population_density_df = pd.read_csv("output.csv")



# states_gdf.head(5)
# xyz_df.head(5)
population_density_df.head(5)[0]



,2.66916667066498992 13.8916668123350107 0
0,2.67000000399497006 13.8916668123350107 0
1,2.67083333732494976 13.8916668123350107 0
2,2.67166667065492947 13.8916668123350107 0
3,2.67250000398490961 13.8916668123350107 0
4,2.67333333731488931 13.8916668123350107 0


In [12]:
population_density_df.iloc[0][0]

'2.67000000399497006 13.8916668123350107 0'

In [ ]:
# Create GeoDataFrame from XYZ
geometry = [Point(xy) for xy in zip(xyz_df['X'], xyz_df['Y'])]
xyz_gdf = gpd.GeoDataFrame(xyz_df, geometry=geometry, crs="EPSG:4326")

# Spatial join: keep only points within the selected states
filtered_gdf = gpd.sjoin(xyz_gdf, states_gdf, how="inner", predicate='within')

# Save filtered points to CSV or GeoJSON
# filtered_gdf[['X', 'Y', 'Z']].to_csv("filtered_points.csv", index=False)

print(f"Filtered {len(filtered_gdf)} points inside the selected states.")
filtered_gdf.head(5)

In [14]:

# Load data for taraba health centers
csv_taraba = "TARABA-PRIMARY HEALTH CENTERS-data - TARABA-PRIMARY HEALTH CENTERS-data.csv"
df_taraba_health_centers = pd.read_csv(csv_taraba)
# Filter operational health centers
taraba_operational_health_centers = df_taraba_health_centers[df_taraba_health_centers["is_operational"] == True]

taraba_operational_health_centers

,state,lga,ward,zone,feedbacks,health_facility_full_name,geo_coordinate,latitude,longitude,altitude,...,facility_screening_services,screening_counts,general_equipment_counts,facility_commodities,data_source,kb_uuid,kb_id,created_at,updated_at,revitalizations
0,TARABA,IBI,DAMPAR I,NORTH EAST,boundary.HealthcareFacilityFeedback.None,Dampar Primary Health Centre,SRID=4326;POINT (10.1342317 8.53486),8.534860,10.134232,93.4,...,NaN,NaN,NaN,NaN,PHASE ONE,b9a55651-5dcc-448b-bba6-5d6b76844bd9,333781717,2025-04-11 12:08:01.067449+00:00,2025-04-14 13:56:51.453115+00:00,Dampar Primary Health Centre - 1
1,TARABA,ARDO KOLA,SUNKANI,NORTH EAST,boundary.HealthcareFacilityFeedback.None,Sunkani Town Clinic,SRID=4326;POINT (11.2619829 8.6978344),8.697834,11.261983,265.5,...,NaN,NaN,NaN,NaN,PHASE ONE,1c925be5-4cef-43e6-bce8-578a68b69fe7,335314201,2025-04-11 12:08:01.067449+00:00,2025-04-14 13:56:52.540239+00:00,Sunkani Town Clinic - 1
2,TARABA,DONGA,FADA,NORTH EAST,boundary.HealthcareFacilityFeedback.None,Gidan-Maimagani Primary Health Centre,SRID=4326;POINT (10.0540785 7.766329),7.766329,10.054079,149.6,...,NaN,NaN,NaN,NaN,PHASE ONE,959f6e7d-1ef8-4657-8278-7211aa9d280e,333970967,2025-04-11 12:08:01.067449+00:00,2025-04-14 13:56:51.531682+00:00,Gidan-Maimagani Primary Health Centre - 1
3,TARABA,SARDAUNA,GEMBU B,NORTH EAST,boundary.HealthcareFacilityFeedback.None,Leme Fuah Health Facility,SRID=4326;POINT (11.2081206 6.7508807),6.750881,11.208121,1547.0,...,NaN,NaN,NaN,NaN,PHASE ONE,5707fda6-be3e-4c92-a154-09b63365fadc,334036288,2025-04-11 12:08:01.067449+00:00,2025-04-14 13:56:51.697031+00:00,Leme Fuah Health Facility - 1
4,TARABA,IBI,DAMPAR III,NORTH EAST,boundary.HealthcareFacilityFeedback.None,Maji Biyu Primary Health Centre,SRID=4326;POINT (9.87297 8.4784717),8.478472,9.872970,130.4,...,NaN,NaN,NaN,NaN,PHASE ONE,96d27994-f3c1-47fe-900f-c83c29ef898b,333547999,2025-04-11 12:08:01.067449+00:00,2025-04-14 13:56:51.470490+00:00,Maji Biyu Primary Health Centre - 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,TARABA,ZING,BUBONG,NORTH EAST,boundary.HealthcareFacilityFeedback.None,Zippo Primary Health Centre,SRID=4326;POINT (11.8040109 8.9587147),8.958715,11.804011,466.0,...,1. Malaria 5. Pregnancy 8. Retroviral Screenin...,"{'malaria': 28, 'diabetes': 0, 'syphilis': 0, ...","{'bed_pan': 0, 'screens': 0, 'camp_gas': 0, 'g...","{'carbetocin': 'No', 'folic_acid': 'No', 'meth...",PHASE 2 BATCH TWO,50c1405f-8a31-442b-8e77-573c9fab36bd,454367616,2025-04-11 12:08:01.067449+00:00,2025-04-14 13:34:03.242393+00:00,No Data
787,TARABA,KARIM LAMIDO,BIKWIN,NORTH EAST,boundary.HealthcareFacilityFeedback.None,Zoh Dutse Health Post,SRID=4326;POINT (11.2197826 9.4507191),9.450719,11.219783,248.6,...,14. None,"{'malaria': 0, 'diabetes': 0, 'syphilis': 0, '...","{'bed_pan': 0, 'screens': 0, 'camp_gas': 0, 'g...","{'carbetocin': 'No', 'folic_acid': 'No', 'meth...",PHASE 2 BATCH TWO,4fdae8c4-a7a5-4b10-838c-f06c9bd4aba2,452000022,2025-04-11 12:08:01.067449+00:00,2025-04-14 13:32:58.063790+00:00,No Data
788,TARABA,KARIM LAMIDO,BIKWIN,NORTH EAST,boundary.HealthcareFacilityFeedback.None,Zoh Makarau Health Post,SRID=4326;POINT (11.2083032 9.4129322),9.412932,11.208303,179.1,...,1. Malaria 5. Pregnancy,"{'malaria': 29, 'diabetes': 0, 'syphilis': 0, ...","{'bed_pan': 1, 'screens': 0, 'camp_gas': 0, 'g...","{'carbetocin': 'No', 'folic_acid': 'Yes', 'met...",PHASE 2 BATCH TWO,267ad91b-2924-4acd-9cd8-64a8dfa556df,452000753,2025-04-11 12:08:01.067449+00:00,2025-04-14 13:32:58.118940+00:00,No Data
789,TARABA,SARDAUNA,KAKARA,NORTH EAST,boundary.HealthcareFacilityFeedback.None,Kakara Maternity Centre,SRID=4326;POINT (11.120024999999998 6.82446669...,6.824467,11.120025,1504.2,...,NaN,NaN,NaN,NaN,PHASE ONE,09910b39-2bf4-48d3-9e91-32bd67f14402,333558983,2025-04-11 12:08:01.067449+00:00,2025-04-14 13:56:51.850726+00:00,Kakara Maternity Centre - 1


In [17]:
# Filter and count
count_per_lga = taraba_operational_health_centers.groupby("lga").size().reset_index(name="operational_facility_count").sort_values(by="operational_facility_count", ascending=False)

count_per_lga

,lga,operational_facility_count
10,SARDAUNA,79
7,KARIM LAMIDO,68
1,BALI,54
4,GASSOL,51
14,YORRO,51
13,WUKARI,50
15,ZING,47
12,USSA,43
0,ARDO KOLA,38
2,DONGA,38


In [23]:
# nga_children_under_five_2020.csv

# Load XYZ data
pop_den_df = pd.read_csv("nga_children_under_five_2020.csv")
pop_den_df.head(20)

,longitude,latitude,nga_children_under_five_2020
0,2.477083,14.000139,3.592445
1,2.509583,14.000139,3.592445
2,2.510139,14.000139,3.592445
3,2.510417,14.000139,3.592445
4,2.510694,14.000139,3.592445
5,2.538472,14.000139,3.592445
6,2.544028,14.000139,3.592445
7,2.544306,14.000139,3.592445
8,3.035694,14.000139,2.470832
9,3.209583,14.000139,2.470832


In [21]:
pop_den_df.columns

Index(['longitude', 'latitude', 'nga_children_under_five_2020'], dtype='object')

In [22]:
pop_den_df.shape

(26592616, 3)

In [ ]:
# Assuming pop_df is your DataFrame with columns: latitude, longitude, etc.

filtered_benue = pop_den_df[
    (pop_den_df['latitude'] >= 6.417) &
    (pop_den_df['latitude'] <= 8.133) &
    (pop_den_df['longitude'] >= 7.783) &
    (pop_den_df['longitude'] <= 10.000)
]

,longitude,latitude,nga_children_under_five_2020
13049291,8.545694,8.132917,1.931165
13049292,8.545972,8.132917,1.931165
13049293,8.546528,8.132917,1.931165
13049294,8.546806,8.132917,1.931165
13049295,8.547083,8.132917,1.931165


In [ ]:
filtered_benue.shape

(1058948, 3)

In [28]:
filtered_benue.to_csv("benue_population_under_five.csv", index=False)

In [29]:
# Load data for benue health centers
csv_benue = "BENUE-PRIMARY HEALTH CENTERS-data - BENUE-PRIMARY HEALTH CENTERS-data.csv"
df_benue_health_centers = pd.read_csv(csv_benue)
# Filter operational health centers
benue_operational_health_centers = df_benue_health_centers[df_benue_health_centers["is_operational"] == True]
count_health_centers_per_lga_benue = benue_operational_health_centers.groupby("lga").size().reset_index(name="operational_facility_count").sort_values(by="operational_facility_count", ascending=False)

# Load data for taraba health centers
csv_taraba = "TARABA-PRIMARY HEALTH CENTERS-data - TARABA-PRIMARY HEALTH CENTERS-data.csv"
df_taraba_health_centers = pd.read_csv(csv_taraba)
# Filter operational health centers
taraba_operational_health_centers = df_taraba_health_centers[df_taraba_health_centers["is_operational"] == True]
count_health_centers_per_lga_tar = taraba_operational_health_centers.groupby("lga").size().reset_index(name="operational_facility_count").sort_values(by="operational_facility_count", ascending=False)

In [30]:
count_health_centers_per_lga_tar.to_csv("oper_health_centers_taraba.csv", index=False)

In [47]:
# taraba_operational_health_centers.head()

taraba_operational_health_centers.head()

columns_to_keep = ["state", "lga","ward","zone","health_facility_full_name","geo_coordinate","latitude","longitude","altitude","is_operational","alternate_facility_name_details","facility_type"]

taraba_operational_health_centers_small = df_taraba_health_centers[columns_to_keep]

taraba_operational_health_centers_small.to_csv("health_centers_taraba_reduced.csv", index=False)

In [51]:
taraba_operational_health_centers.lga.unique()

array(['IBI', 'ARDO KOLA', 'DONGA', 'SARDAUNA', 'TAKUM', 'GASHAKA',
       'KARIM LAMIDO', 'USSA', 'KURMI', 'BALI', 'JALINGO', 'LAU', 'ZING',
       'YORRO', 'WUKARI', 'GASSOL'], dtype=object)

In [48]:
# taraba_operational_health_centers.head()

df_benue_health_centers.head()

columns_to_keep = ["state", "lga","ward","zone","health_facility_full_name","geo_coordinate","latitude","longitude","altitude","is_operational","alternate_facility_name_details","facility_type"]

benue_operational_health_centers_small = df_benue_health_centers[columns_to_keep]

benue_operational_health_centers_small.to_csv("health_centers_benue_reduced.csv", index=False)

In [52]:
benue_operational_health_centers.lga.unique()

array(['ADO', 'AGATU', 'APA', 'BURUKU', 'GBOKO', 'GUMA', 'GWER EAST',
       'GWER WEST', 'KATSINA ALA', 'KONSHISHA', 'KWANDE', 'LOGO',
       'MAKURDI', 'OBI', 'OGBADIBO', 'OHIMINI', 'OJU', 'OKPOKWU',
       'OTUKPO', 'TARKA', 'UKUM', 'USHONGO', 'VANDEIKYA'], dtype=object)